In [ ]:
! pip install pymssql
! pip install  SQLAlchemy==1.4.17

##Growth and Profitability 150/50 Strategy

This strategy uses indicators of growth and profitability to select stocks, aiming to capitalize on the performance of financially robust companies while shorting those with poorer prospects.

**Selected Features:**
- **`egr`**: Growth in common shareholder equity, to identify companies that are expanding their equity base and potentially their business.
- **`gma`**: Gross profitability, a profitability measure that focuses on the gross profits made relative to assets.
- **`roic`**: Return on invested capital, a comprehensive profitability ratio that measures how well a company is using its capital to generate profits.
- **`sgr`**: Sales growth, which can signal increasing market demand or successful business expansion.
- **`ep`**: Earnings to price ratio, an inverse of the P/E ratio, showing how much earnings a company produces per unit of stock price.

This version of the 150/50 strategy focuses on leveraging momentum indicators combined with volatility measures to choose stocks

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import QuantileTransformer, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from pandas_datareader.data import DataReader
import statsmodels.formula.api as smf
import plotly.graph_objects as go


In [ ]:
# Database Connection Setup
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912"
database = "ghz"
connection_string = f"mssql+pymssql://{username}:{password}@{server}/{database}"
conn = create_engine(connection_string).connect()

In [ ]:
# Data Retrieval
query = """
SELECT date, ticker, egr, gma, roic, sgr, ep, siccd, ret
FROM data
ORDER BY date, ticker
"""
df = pd.read_sql(query, conn)
df['date'] = pd.to_datetime(df['date'])
df = df.dropna()
df.set_index(['date', 'ticker'], inplace=True)

df.head(5)

egr       gma      roic       sgr        ep  siccd  \
date       ticker                                                            
2000-01-01 AA      0.375034  0.259567  0.095845  0.151706  0.062376   3334   
           AAC    -0.227432 -0.027672 -0.061952  0.081011 -0.586435   6153   
           AACE    0.239352  0.201136  0.211095  0.220772  0.058082   6090   
           AAG     0.179483  0.021361  0.021609  0.110357  0.099564   3675   
           AAGP   -0.038661  0.814228 -0.003277 -0.100378 -0.011439   5130   

                        ret  
date       ticker            
2000-01-01 AA     -0.160392  
           AAC     0.056338  
           AACE   -0.067568  
           AAG    -0.100694  
           AAGP    0.364865

In [ ]:
# Feature Engineering
qt = QuantileTransformer(output_distribution='normal')
features = ["egr", "gma", "roic", "sgr", "ep"]
df["actual"] = df.ret

grouped = df.groupby("date", group_keys=False)
df[features+["ret"]] = grouped[features+["ret"]].apply(
  lambda d:
    pd.DataFrame(
      qt.fit_transform(d),
      columns=d.columns,
      index=d.index
    )
)


In [ ]:
# get industry
inds = pd.read_csv("siccodes12.csv", index_col="industry")
ind_names = inds.index.unique().to_list()

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"

codes = pd.Series({code: industry(code) for code in df.siccd.unique()})
codes = pd.DataFrame(codes).reset_index()
codes.columns = ["siccd", "industry"]

df = df.reset_index().merge(codes, on="siccd")
df = df.set_index(["date", "ticker"])

features.append("industry")

df

egr       gma      roic       sgr        ep  siccd  \
date       ticker                                                            
2000-01-01 AA      1.012516 -0.226209  0.397801  0.193140  0.479836   3334   
           KLU    -0.296174 -0.722693 -0.078923 -0.864455 -0.466562   3334   
           MXM     2.909544 -0.765457 -0.236691 -0.890835 -0.653548   3334   
2000-02-01 AA      1.013944 -0.222337  0.398127  0.200649  0.476747   3334   
           KLU    -0.292435 -0.719403 -0.077064 -0.857820 -0.466282   3334   
...                     ...       ...       ...       ...       ...    ...   
2022-02-01 LDOS    0.446920  0.058719  0.548461  0.711525  0.365108   9711   
2022-03-01 LDOS    0.438608  0.053291  0.545067  0.699735  0.359140   9711   
2021-03-01 SYX     1.089342  1.269408  1.545798  0.096828  0.411522   5046   
2021-04-01 SYX     1.088730  1.270120  1.567757  0.122126  0.415186   5046   
2021-05-01 SYX     1.081548  1.273929  1.568736  0.129504  0.414362   5046   

                        ret    actual       industry  
date       ticker                                     
2000-01-01 AA     -1.147688 -0.160392  Manufacturing  
           KLU    -1.558599 -0.219512  Manufacturing  
           MXM    -1.311458 -0.182216  Manufacturing  
2000-02-01 AA     -0.201181 -0.013453  Manufacturing  
           KLU    -0.479117 -0.052083  Manufacturing  
...                     ...       ...            ...  
2022-02-01 LDOS    1.258672  0.138513          Other  
2022-03-01 LDOS    0.606024  0.064218          Other  
2021-03-01 SYX     1.080115  0.143490          Shops  
2021-04-01 SYX     0.246033  0.038911          Shops  
2021-05-01 SYX    -1.918155 -0.192884          Shops  

[963919 rows x 9 columns]

In [ ]:
# Define the Model
transform1 = make_column_transformer(
    (OneHotEncoder(), ["industry"]),
    remainder="passthrough"
)
transform2 = PolynomialFeatures(degree=2)
model = LinearRegression(fit_intercept=False)
pipe = make_pipeline(transform1, transform2, model)

In [ ]:
dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
predictions = None

for train_date, end_date in zip(dates[:-1], dates[1:]):

  fltr1 = df.index.get_level_values("date") < train_date
  fltr2 = df.index.get_level_values("date") < end_date
  train = df[fltr1]
  test = df[~fltr1 & fltr2]

  Xtrain = train[features]
  ytrain = train["ret"]
  Xtest = test[features]
  ytest = test["ret"]

  pipe.fit(Xtrain, ytrain)
  print('Train set score: ' + str(pipe.score(Xtrain, ytrain)))

  pred = pipe.predict(Xtest)
  pred = pd.Series(pred, index=test.index)
  predictions = pd.concat((predictions, pred))
  print('Test set score: ' + str(pipe.score(Xtest,ytest)))

predict_df = pd.DataFrame({'pred_ret':predictions})



Train set score: 0.009124031661802201
Test set score: -0.0012098829636590924
Train set score: 0.0062118803452212434
Test set score: 0.0028435900983331264
Train set score: 0.0056408825552534125
Test set score: 0.006119166534422571
Train set score: 0.005979395949691879
Test set score: 0.0034437857700387875


In [ ]:
def get_stock_returns(predict_df, actual_returns_df, num_stocks, best=True):
    """Function to get the mean returns of the best or worst stocks based on prediction."""
    # Group by date and rank stocks based on their predicted returns
    grouped = predict_df.groupby("date", group_keys=False)
    if best:
        # If best is True, rank descending and pick the top stocks
        ranks = grouped.rank(ascending=False, method="first")
    else:
        # Otherwise, rank ascending and pick the bottom stocks
        ranks = grouped.rank(ascending=True, method="first")

    # Select stocks based on the ranks
    selected_stocks = predict_df[ranks <= num_stocks].dropna()

    # Merge with actual returns to calculate the returns
    actual_ret = pd.concat([selected_stocks, actual_returns_df], axis=1, join='inner')
    actual_ret.columns = ["pred_ret", "ret"]

    # Calculate average returns by date
    mean_rets = actual_ret.groupby("date").ret.mean()

    # Format the index to display just year and month
    mean_rets.index = pd.to_datetime(mean_rets.index)
    mean_rets.index = mean_rets.index.strftime('%Y-%m')

    return mean_rets

# Assuming predict_df contains the prediction and df['actual'] contains actual returns
long_num_stocks = 100
short_num_stocks = 100

# Get the best and worst stock returns
best_rets = get_stock_returns(predict_df, df["actual"], long_num_stocks, best=True)
worst_rets = get_stock_returns(predict_df, df["actual"], short_num_stocks, best=False)

In [ ]:
import yfinance as yf
from pandas.tseries.offsets import BMonthEnd

import datetime as dt


spy = yf.download("SPY", start=2017)["Adj Close"]
spy = pd.DataFrame(spy)
spy_monthly = spy.resample('M').last().pct_change()
spy_monthly.index = spy_monthly.index.strftime('%Y-%m')
spy_monthly.columns = ['Monthly Returns']

rets = pd.concat([spy_monthly, best_rets, worst_rets], axis=1, join='inner')
rets.columns = ["spy", "best", "worst"]

rets["150/50"] = 1.5*rets.best - 0.5*rets.worst
rets

[*********************100%%**********************]  1 of 1 completed


,spy,best,worst,150/50
2005-01,-0.022421,0.010642,-0.109315,0.070620
2005-02,0.020904,0.025518,-0.023432,0.049993
2005-03,-0.018293,-0.024365,-0.092346,0.009626
2005-04,-0.018735,-0.036846,-0.097063,-0.006737
2005-05,0.032225,0.037119,0.077820,0.016769
...,...,...,...,...
2021-11,-0.008035,-0.028039,-0.120876,0.018379
2021-12,0.046248,0.075004,-0.076643,0.150828
2022-01,-0.052741,-0.033259,-0.118353,0.009287
2022-02,-0.029517,0.004110,-0.005321,0.008826


In [ ]:
from pandas_datareader import DataReader as pdr


ff = pdr("F-F_Research_Data_Factors", "famafrench", start=2005)[0]/100
ff.index = ff.index.astype(str)
mkt = ff["Mkt-RF"] + ff["RF"]
rf = ff["RF"]

df_sharpe = pd.concat((rets['150/50'], mkt, rf), axis=1)
df_sharpe.columns = ["ret", "mkt", "rf"]

<ipython-input-57-954bc635dc37>:4: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.

<ipython-input-57-954bc635dc37>:4: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.



In [ ]:
xret = df_sharpe.ret - df_sharpe.rf
print(f"Annualized Sharpe ratio of Growth and Profitability 150/50 Strategy is {np.sqrt(12)*xret.mean()/xret.std():.2%}")
print(f'Mean return of Growth and Profitability 150/50 Strategy is, {xret.mean():.4}')
print(f'Std dev of Growth and Profitability 150/50 Strategy is, {xret.std():.4}')

Annualized Sharpe ratio of Growth and Profitability 150/50 Strategy is 42.73%
Mean return of Growth and Profitability 150/50 Strategy is, 0.00809
Std dev of Growth and Profitability 150/50 Strategy is, 0.06559


In [ ]:
import plotly.graph_objects as go


trace1 = go.Scatter(
    x = rets.index.to_list(),
    y = (1+rets['150/50']).cumprod(),
    mode="lines",
    name="150-50 strategy return",
    hovertemplate="%{x}<br>accumulation = $%{y:.2f}<extra></extra>"
)

trace2 = go.Scatter(
    x = rets.index.to_list(),
    y = (1+rets['spy']).cumprod(),
    mode="lines",
    name="SPY 500 return",
    hovertemplate="%{x}<br>accumulation = $%{y:.2f}<extra></extra>"
)

fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Comparison of Growth and Profitability 150/50 Strategy vs SPY 500 Returns",
    yaxis_title="",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickprefix="$",
    yaxis_tickformat=".2f",
    height=600,
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)
fig.show()

In [ ]:
# Jensen's alpha
ff = pdr("F-F_Research_Data_Factors", "famafrench", start=2005)[0]/100
ff.index = ff.index.astype(str)
mkt_rf = ff["Mkt-RF"]
rf = ff["RF"]

df_ja = pd.concat((rets['150/50'], mkt_rf, rf), axis=1)
df_ja.columns = ["ret", "mkt_rf", "rf"]
df_ja["ret_rf"] = df_ja.ret - df_ja.rf
df_ja["mkt"] = df_ja.mkt_rf + df_ja.rf
df_ja.index.name = "date"
df_ja = df_ja.reset_index()
df_ja = df_ja.dropna()

import statsmodels.formula.api as smf


result = smf.ols("ret_rf~mkt_rf", df_ja).fit()
beta = result.params["mkt_rf"]
mkt = df_ja.rf + beta*df_ja.mkt_rf
active = df_ja.ret - mkt

# Extracting the alpha (intercept) from the model summary
#This value represents the Jensen's  alpha for the 150/50 portfolio, indicating its performance above the expected return based on the Fama-French factors model
alpha_ja = result.params['Intercept']
print(f'Jensen\'s alpha of Growth and Profitability 150/50 Strategy is {alpha_ja:.3}')

Jensen's alpha of Growth and Profitability 150/50 Strategy is 0.000274


<ipython-input-60-84c1bb1beeb6>:2: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.

<ipython-input-60-84c1bb1beeb6>:2: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.



In [ ]:
trace1 = go.Scatter(
    x = df_ja.date,
    y = (1+df_ja.ret).cumprod(),
    name = "150-50 strategy total",
    hovertemplate="%{x}<br>total=$%{y:.2f}<extra></extra>"
)
trace2 = go.Scatter(
    x = df_ja.date,
    y = (1+active).cumprod(),
    name = "150-50 strategy active",
    hovertemplate="%{x}<br>active=$%{y:.2f}<extra></extra>"
)
trace3 = go.Scatter(
    x = df_ja.date,
    y = (1+mkt).cumprod(),
    name = "150-50 strategy market",
    hovertemplate="%{x}<br>market=$%{y:.2f}<extra></extra>"
)
fig = go.Figure()
for trace in [trace1, trace3, trace2]:
    fig.add_trace(trace)

fig.update_layout(
    title="Growth and Profitability 150/50 Strategy Accumulation",
    yaxis_title="Accumulation",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickprefix="$",
    yaxis_tickformat=".2f",
    height=600,
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)

fig.show()

In [ ]:
# Fama-French alpha
df_ff = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100
df_ff.index = df_ff.index.astype(str)
df_ff["ret"] = rets['150/50']
df_ff["ret_rf"] = df_ff.ret - df_ff.RF
df_ff = df_ff.dropna()
df_ff.index = df_ff.index.astype(str)
df_ff.index.name = "date"
df_ff = df_ff.reset_index()
df_ff = df_ff.rename(columns={"Mkt-RF": "mkt_rf", "RF": "rf"})

result_ff = smf.ols("ret_rf~mkt_rf+SMB+HML+CMA+RMW", df_ff).fit()
betas_ff = result_ff.params[1:]
mkt_ff = df_ff.rf + betas_ff[0]*df_ff.mkt_rf
smb = betas_ff[1]*df_ff.SMB
hml = betas_ff[2]*df_ff.HML
cma = betas_ff[3]*df_ff.CMA
rmw = betas_ff[4]*df_ff.RMW
active_ff = df_ff.ret - mkt_ff - smb - hml - cma - rmw

# Extracting the alpha (intercept) from the model summary
#This value represents the Fama-French alpha for the 150/50 portfolio, indicating its performance above the expected return based on the Fama-French factors model
alpha_ff = result_ff.params['Intercept']
print(f'Fama-French alpha of Growth and Profitability 150/50 Strateg is {alpha_ff:.3}')

Fama-French alpha of Growth and Profitability 150/50 Strateg is 3.53e-05


<ipython-input-63-2c17c8b3b07c>:2: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.

<ipython-input-63-2c17c8b3b07c>:2: FutureWarning:

The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.

